In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
import matplotlib.pyplot as plt




In [2]:
class dataset(Dataset):

    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y= torch.tensor(y, dtype=torch.float32)
        self.length = self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


    def __len__(self):
        return self.length


In [ ]:
### don't run this, was trying to fix a Pandas warning
drugs = pd.read_excel("drug_dataset.xlsx")


drugs.loc[drugs.Heroin == "CL2"]

drug = "Heroin"

cols = ["Nscore", "Escore", "Oscore", "Ascore", drug]
subset = drugs[cols]
drug_use = subset[drug]

non_user = ["CL0", "01"]
user = ["CL" + str(i) for i in range(2, 7)]
class_ = [0, 1]

consumer_conditions = [(subset.loc[subset[drug].isin(non_user)]),(subset.loc[subset[drug].isin(user)])]
#consum_conditions = [(drug_use.isin(non_user)), (drug_use.isin(user))]

subset[drug] = np.select(consumer_conditions, class_)



In [4]:
def create_sub(df, drug):
    cols = ["Nscore", "Escore", "Oscore", "Ascore", drug]
    subset = df[cols]
    drug_use = subset[drug]

    non_user = ["CL0", "01"]
    user = ["CL" + str(i) for i in range(2, 7)]
    class_ = [0, 1]

    consum_conditions = [(drug_use.isin(non_user)), (drug_use.isin(user))]

    subset[drug] = np.select(consum_conditions, class_)

    return subset


#Heroin subset
heroin_sub = create_sub(drugs, "Heroin")


/var/folders/5w/zg23p5bd3bnc86d75_pkn99m0000gn/T/ipykernel_59411/4224102323.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset[drug] = np.select(consum_conditions, class_)


In [5]:

X = heroin_sub.iloc[:,:-1]

y = heroin_sub.iloc[:, -1:] 

test_size = 0.2
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)


In [10]:
pd.concat([x_train, y_train], axis=1)

,Nscore,Escore,Oscore,Ascore,Heroin
840,-0.67825,0.00332,0.29338,1.81866,0
1317,0.82562,0.96248,-0.17779,-0.30172,0
1385,0.62967,0.47617,0.44585,-0.60633,1
347,-0.58016,0.32197,-0.45174,-0.15487,0
602,-0.67825,-0.80615,-0.45174,-1.47955,1
...,...,...,...,...,...
835,-0.34799,0.32197,0.14143,-1.21213,1
1216,0.31287,1.28610,-0.17779,-0.76096,1
1653,1.72012,-1.23177,-0.71727,-0.60633,0
559,1.02119,0.16767,-0.71727,0.28783,0


In [6]:
# splitting data into training and testing set
def split_data(df, test_size=0.2):

    X = df.iloc[:,:-1]
    y = df.iloc[:, -1:]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)

    data = [x_train, y_train, x_test, y_test]
    data = list(map(lambda df: df.to_numpy(), data))

    x_train, y_train, x_test, y_test = data

    trainset = dataset(x_train, y_train)
    testset = dataset(x_test, y_test)


    #train_set = pd.concat([x_train, y_train], axis=1)
    #test_set = pd.concat([x_test, y_test], axis=1)

    trainloader = DataLoader(trainset, batch_size=5, shuffle=False)
    testloader = DataLoader(testset, batch_size=5, shuffle=False)

    

    return trainloader, testloader









X = heroin_sub.iloc[:,:-1].to_numpy()

y = heroin_sub.iloc[:, -1:].to_numpy()


trainloader, testloader = split_data(heroin_sub)









In [7]:
class Net(nn.Module):

    def __init__(self, input_shape) -> None:
        super(Net, self).__init__()

        self.fc1 = nn.Linear(input_shape, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 1)

    
    def forward(self, x):

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))


        return x



learning_rate = 0.01
epochs = 700

model = Net(input_shape=X.shape[1])
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.BCELoss()



    

In [8]:
losses = []
accur = []

for i in range(epochs):
    for j, (x_train, y_train) in enumerate(trainloader):

        output = model(x_train)

        loss = loss_fn(output, y_train.reshape(-1, 1))

        predicted = model(torch.tensor(X, dtype=torch.float32))
        acc = (predicted.reshape(-1).detach().numpy().round() == 1).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    if i % 50 == 0:
        losses.append(loss)
        accur.append(acc)

        print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))



epoch 0	loss : 0.2799536883831024	 accuracy : 0.0
epoch 50	loss : 0.2681226432323456	 accuracy : 0.002652519893899204
epoch 100	loss : 0.27550008893013	 accuracy : 0.01220159151193634
epoch 150	loss : 0.29198580980300903	 accuracy : 0.028116710875331564
epoch 200	loss : 0.3265521824359894	 accuracy : 0.05570291777188329
epoch 250	loss : 0.36768683791160583	 accuracy : 0.09071618037135279
epoch 300	loss : 0.4047579765319824	 accuracy : 0.12519893899204243
epoch 350	loss : 0.4292100965976715	 accuracy : 0.1570291777188329
epoch 400	loss : 0.42330703139305115	 accuracy : 0.16976127320954906
epoch 450	loss : 0.4505179226398468	 accuracy : 0.1830238726790451
epoch 500	loss : 0.454973429441452	 accuracy : 0.1909814323607427
epoch 550	loss : 0.451212078332901	 accuracy : 0.19681697612732096
epoch 600	loss : 0.43354764580726624	 accuracy : 0.21114058355437665
epoch 650	loss : 0.3874434232711792	 accuracy : 0.21220159151193635


: 